In [1]:
(ns push4-clj.notebook
    (:require [clojure.spec.alpha :as s]
              [clojure.pprint :as pp]
              [push4-clj.expression :as expr]
              [push4-clj.spec-utils :as su]
              [push4-clj.dag :as dag]
              [push4-clj.push :as push]))

# Push4 (Name TBD) Overview

The goals of this Push system are:

1. Evolve programs that utilize the pre-existing functions instead of hand-written instruction sets.
2. Evolve programs that can use any and all (immutable) data types.
3. Produce source code in the host language (Clojure) as an output.

This version of Push differs from Push3 (Clojush) in a number of ways:

1. Push execution produces the evolved, executable program. In a sense, Push is acting as a compiler.
2. The input to the system is a set of **expressions** that consist of some `code` and some `spec`.
    1. The `code` can be a function, literal, or input.
    2. The `spec` is a Clojure spec that describes the `code`. This is used to build programs free of runtime errors.
3. There is only one stack in the Push state. It holds DAGs of expressions.

This notebook demonstrates the following

1. The setup required. Mainly, defining expressions.
2. The compilation of a linear Push program to a DAG.
3. The execution of the DAG as a program.
4. The translation of the DAG to Clojure source code.

The prototype library (`push4-clj`) used by this notebook can be found [here](https://github.com/erp12/push4-clj-prototype). It clocks in at under 350 lines of code.

In [2]:
;; To start, we define some specs that wrap simple Clojure predicates.

(def number?-spec
  (s/spec number?))


(def int?-spec
  (s/spec int?))


(def seq?-spec
  (s/spec seq?))

#'push4-clj.notebook/seq?-spec

A Push language implementation should obey the law of **no runtime errors**. To accomplish this while still allowing an arbitrary set of functions to appear in the genomes/programs, we need to understand which functions produce values that can be safely used as arguments to other functions.

This is the job of a function spec. `clojure.spec` already has function specs that are a composition of other specs for the individual arguments and return value. Ideally, our Push system would accept these specs and extract the individual arg specs and return spec in order to understand which functions can be called to produce a valid argument to another function. Unfortunately, I could not find a way to extract individual argument specs from the existing function specs.


### Deconstructed Function Spec (DFS)

To solve this problem, we use a "deconstructed function spec" map that simply holds 1) a vector of specs for the function's args and 2) a single spec for the function's return value. A DFS definition for the `conj` function might look something like this:

```clojure
{::arg-specs   [(s/spec coll?) (s/spec any?)]
 ::return-spec (s/spec coll?)}
```

### Expressions

An expression is simply a map that wraps some code that produces a value. An expression can wrap a constant value, a function, or a symbol/position corresponding to one of the inputs that will be supplied to the evolved program.

Expressions that wrap constant values are called literal expressions. They only require the literal value to be created.

Expressions that wrap a function require a name/symbol, a DFS, and the function object.

Expressions that wrap a program input require a name/symbol, a spec, and the position the input falls within the argument list.

In [3]:
;; This DFS describes all functions that take two numbers and return a single number.

(def simple-numeric-fn-dfs
  (expr/make-dfs [number?-spec number?-spec] number?-spec))

#'push4-clj.notebook/simple-numeric-fn-dfs

In [4]:
;; A map of all epxressions we would provide the PushGP system. We are using a small set for the demo.
;; We give each expression a keyword for convinience. 
(def expressions
  {:+      (expr/fn->expression '+ + simple-numeric-fn-dfs)
   :-      (expr/fn->expression '- - simple-numeric-fn-dfs)
   :*      (expr/fn->expression '* * simple-numeric-fn-dfs)
   :/      (expr/fn->expression '/ / simple-numeric-fn-dfs)
   :int    (expr/fn->expression 'int int (expr/make-dfs [number?-spec] int?-spec))
   :take   (expr/fn->expression 'take take (expr/make-dfs [int?-spec seq?-spec] seq?-spec))
   :count  (expr/fn->expression 'count count (expr/make-dfs [seq?-spec] (s/spec nat-int?)))
   :2      (expr/lit->expression 2)
   :10     (expr/lit->expression 10)
   :my-seq (expr/sym-expression 'my-seq seq?-spec 0)})

#'push4-clj.notebook/expressions

After defining all the expressions, we can find the set of unique specs that exist throughout all function arguments, function return types, and program inputs. This is called the `spec-set`.

Next, we need to understand which specs are strictly broader than other specs. The best term I could think of is spec "dominance". Spec `A` "dominates" spec `B` if all valid values of spec `B` are also valid values of spec `A`. Using this information, we can produce programs with no runtime errors (assuming all functions are correctly spec-ed).
If we know that $S_2$ dominates $S_1$, then any function which requires a valid $S_2$ argument can get its argument from the returned value of a function which produces a valid $S_1$.

As far as I am aware, there is no 100% accurate way to compare two specs and determine if one dominates the other. It is, however, possible to make a good guess by using `clojure.spec.gen` generator capabilities. Given a spec, $S_1$ we can generate a large sample of valid values (I used 1000). If every element of the sample is a valid value of another spec, $S_2$, then we can assume that $S_2$ dominates $S_1$. 

In [5]:
(def spec-dominance-map
  (su/spec-dominance (expr/expressions->spec-set (vals expressions))))

;; Spec-set
;;   number?
;;   int?
;;   nat-int?
;;   seq?

;; Spec Dominance
;;   number? dominates [int? nat-int?]
;;   int?    dominates [nat-int?]

#'push4-clj.notebook/spec-dominance-map

For this demo, our "genome" will be the following sequence of expressions. We will be evaluating this genome as a Push program, however in future versions of this Push system we would want to do the typical translation from linear genome to a nested Push program.

In addition to expressions, nested Push programs may contain stack instructions (dup, rot, reverse, etc) and exec instruction (exec_dup, exec_rot, etc). These instructions must not take any arguments other than the stack and/or the remaining program being evaluated.

**For this demo, we will be producing a program that takes the first half a sequence and drops the rest.**

In [6]:
(def take-first-half-genome
  (map expressions
       [:2 :my-seq :count :/ :int :my-seq :take]))

#'push4-clj.notebook/take-first-half-genome

### Push as a compiler

The goal of Push genomee/program execution is **not** to run the evolved program on a set of inputs. Instead, Push execution does not take any arguments and produces a DAG of expressions that can be used as a program. 

There is no need for control flow in the Push program. Iteration may be used to create redundant parts of the DAG. Duplicating expressions may be done as a way to re-use the output of one expression as the input to multiple other expressions.

### One Stack

In a typical PushGP system, there is ambiguity as to which stack to pop an argument from **when there is any overlap/heirachy/inheritance of data types**. If we want to scale to "real-world" applications and/or integrate with existing code-bases, we will encounter overlapping types.

Functions which can take integers as arguments can, by definition, take their arguments from functions that produce strictly positive integers. A function which takes natural integers *cannot* safely take a value from a function that produces integers.

There are multiple strategies to fix this, but I (arbitrarily) chose to collapse the entire Push state down to one stack. When processing an expression from the Push program, its argument (aka child) expressions are found by starting from the top of the stack and traversing down each element. Using the spec "dominance" information we computed above, we can check if the stack element produces a value of the argument spec *or* a narrower spec that will be guarenteed not to cause runtime errors. If no, we move down the stack one item.

Once the entire Push program has been evaluated, we traverse the stack once more. This time we are looking for the top-most expression DAG whose return spec is the return spec for our evoled program or a narrower spec.

In [7]:
(def program
    (push/compile-to-dag take-first-half-genome  ;; Genome (linear Push program) to conver to a DAG
                         seq?-spec               ;; We would like the DAG (evolved program) to prodce a Seq.
                         spec-dominance-map))    ;; The spec dominace info computeed above.

#'push4-clj.notebook/program

After "compiling" the program, we can evaluate it on some input sequences. This is how we would perform evaluation during evolution. This does not require re-running Push.

In [18]:
(println "First half of 20 element seq\t" (dag/eval-dag program [(range 20)]))
(println "First half of 4 element vec\t"  (dag/eval-dag program [[:A :B :C :D]]))
(println "First half of empty list\t"     (dag/eval-dag program ['()]))

First half of 20 element seq	 (0 1 2 3 4 5 6 7 8 9)
First half of 4 element vec	 (:A :B)
First half of empty list	 ()


### Producing Clojure Code

We can convert the DAG to Clojure code.

In [9]:
(println (dag/dag->code program))

(take (int (/ (count my-seq) 2)) my-seq)


We can also supply a function name/symbol and some argument names to produce an entire function `defn`.

In [10]:
(pp/pprint (dag/dag->defn 'take-first-half ['my-seq] program))

(clojure.core/defn
 take-first-half
 [my-seq]
 (take (int (/ (count my-seq) 2)) my-seq))


I don't have a clever way to get the formatting right, but let's try it out!

In [11]:
(clojure.core/defn
 take-first-half
 [my-seq]
 (take (int (/ (count my-seq) 2)) my-seq))

#'push4-clj.notebook/take-first-half

In [12]:
(take-first-half [:a :b :c :d])

(:a :b)

In [13]:
(take-first-half '())

()

In [14]:
(take-first-half (range 10))

(0 1 2 3 4)